In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

# import folium
import numpy as np
import random
import json
import lightgbm

In [22]:
df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [23]:
sorted(df.columns)

['.geo',
 'area',
 'crop',
 'id',
 'nd_mean_2021-04-15',
 'nd_mean_2021-04-16',
 'nd_mean_2021-04-18',
 'nd_mean_2021-04-19',
 'nd_mean_2021-04-20',
 'nd_mean_2021-04-22',
 'nd_mean_2021-04-23',
 'nd_mean_2021-04-25',
 'nd_mean_2021-04-26',
 'nd_mean_2021-04-27',
 'nd_mean_2021-04-28',
 'nd_mean_2021-04-29',
 'nd_mean_2021-04-30',
 'nd_mean_2021-05-01',
 'nd_mean_2021-05-02',
 'nd_mean_2021-05-03',
 'nd_mean_2021-05-04',
 'nd_mean_2021-05-07',
 'nd_mean_2021-05-08',
 'nd_mean_2021-05-09',
 'nd_mean_2021-05-10',
 'nd_mean_2021-05-15',
 'nd_mean_2021-05-16',
 'nd_mean_2021-05-17',
 'nd_mean_2021-05-19',
 'nd_mean_2021-05-20',
 'nd_mean_2021-05-21',
 'nd_mean_2021-05-24',
 'nd_mean_2021-05-26',
 'nd_mean_2021-05-27',
 'nd_mean_2021-05-29',
 'nd_mean_2021-06-02',
 'nd_mean_2021-06-03',
 'nd_mean_2021-06-04',
 'nd_mean_2021-06-05',
 'nd_mean_2021-06-06',
 'nd_mean_2021-06-07',
 'nd_mean_2021-06-09',
 'nd_mean_2021-06-10',
 'nd_mean_2021-06-12',
 'nd_mean_2021-06-13',
 'nd_mean_2021-06-16',


In [24]:
df['geo_dict'] = df['.geo'].apply(lambda x: json.loads(x))
test_df['geo_dict'] = test_df['.geo'].apply(lambda x: json.loads(x))

In [25]:
df.tail(3)

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,...,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop,geo_dict
4827,5226,58,0.191255,0.063581,0.140111,0.323984,0.000000,0.005056,0.092693,0.399439,...,0.187352,0.142930,0.000000,0.047572,0.007225,0.145505,0.051215,0.079730,6,"{'type': 'Polygon', 'coordinates': [[[38.57789..."
4828,5390,14,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,...,0.644357,0.085861,0.631304,0.694524,0.031836,0.453151,0.000000,0.006495,5,"{'type': 'Polygon', 'coordinates': [[[53.31082..."
4829,860,75,0.226925,0.043076,0.058611,0.440539,0.120169,0.138846,0.020173,0.117669,...,0.003272,0.000000,0.010312,0.227649,0.000000,0.096585,0.000000,0.011459,6,"{'type': 'Polygon', 'coordinates': [[[38.79371..."


In [26]:
df['geo_dict'].apply(lambda x: x['type']).value_counts()

Polygon               4801
MultiPolygon            19
GeometryCollection      10
Name: geo_dict, dtype: int64

# add centroid coords

In [27]:
from shapely.geometry import shape

df['x_centroid'] = df['geo_dict'].apply(lambda x: shape(x).centroid.x)
df['y_centroid'] = df['geo_dict'].apply(lambda x: shape(x).centroid.y)

test_df['x_centroid'] = test_df['geo_dict'].apply(lambda x: shape(x).centroid.x)
test_df['y_centroid'] = test_df['geo_dict'].apply(lambda x: shape(x).centroid.y)

In [28]:
df.tail(2)

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,...,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop,geo_dict,x_centroid,y_centroid
4828,5390,14,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,...,0.631304,0.694524,0.031836,0.453151,0.0,0.006495,5,"{'type': 'Polygon', 'coordinates': [[[53.31082...",53.320243,55.183351
4829,860,75,0.226925,0.043076,0.058611,0.440539,0.120169,0.138846,0.020173,0.117669,...,0.010312,0.227649,0.000000,0.096585,0.0,0.011459,6,"{'type': 'Polygon', 'coordinates': [[[38.79371...",38.801961,52.728333


# vis

In [13]:
def add_points(data_, style):
    for _, r in data_.iterrows():
        geo_j = r["geo_dict"]
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: style)
        geo_j.add_to(m)
        
#         if random.random() < 0.1:
#             folium.Marker(location=[r['y_centroid'], r['x_centroid']],
#                       popup='area: {}'.format(r['area'])).add_to(m)

In [76]:
styles = [{"fillColor": "red", 'color': "red"}, #crop 0
         {"fillColor": "blue", 'color': "blue"},
         {"fillColor": "green", 'color': "green"},
         {"fillColor": "black", 'color': "black"},
         {"fillColor": "pink", 'color': "pink"},
         {"fillColor": "yellow", 'color': "yellow"},
         {"fillColor": "orange", 'color': "orange"}, #crop 6
         {"fillColor": "gray", 'color': "gray"},] # test

In [78]:
# m = folium.Map(location=[53.2, 50.1], zoom_start=5)

# for crop in df.crop.unique():
#     df_crop = df[df.crop == crop]
#     add_points(df_crop, styles[crop])

# add_points(test_df, styles[-1])

# m

In [11]:
df.shape, test_df.shape

((4830, 77), (2071, 76))

In [29]:
df.drop(columns='geo_dict').to_csv('train+.csv', index=False)
test_df.drop(columns='geo_dict').to_csv('test+.csv', index=False)